In [1]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

# check environment
import os
print(f'Conda Environment: ' + os.environ['CONDA_DEFAULT_ENV'])

from platform import python_version
print(f'python version: {python_version()}')

Conda Environment: ROICaT
python version: 3.9.13


In [2]:
# import pprint
from pathlib import Path
# import re
from functools import partial
import gc

# import natsort
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
# import itertools
# import datetime

import scipy.io
import pandas as pd
import numpy as np
# import matplotlib as mpl
import torch

In [4]:
%load_ext autoreload
%autoreload 2
import bnpm
import bnpm.video
import bnpm.server
import bnpm.plotting_helpers
import bnpm.file_helpers
import bnpm.path_helpers
import bnpm.indexing
import bnpm.timeSeries
import bnpm.decomposition

import bnpm.parallel_helpers

In [5]:
mouse = 'mouse_1511L'
cage = 'BMI_cage_1511_4'

In [18]:
## mouse_1511L

dates_toUse = [
#     '20230111',
    '20230112',
    '20230113',
    '20230114',
    '20230115',
    '20230116',
    '20230117',
    '20230118',
    '20230119',
]

In [19]:
date = '20230119'

In [20]:
Fs = 30

In [21]:
paths_local_loggerAligned = bnpm.path_helpers.find_paths(
    dir_outer=f'/media/rich/bigSSD/downloads_tmp/tmp_data/{mouse}/loggers/',
    reMatch='logger_aligned.mat',
    depth=2
)
paths_local_loggerAligned = [p for p in paths_local_loggerAligned if Path(p).parent.name in dates_toUse]
display(paths_local_loggerAligned)

load_loggerAligned = lambda path: {Path(path).parent.name: scipy.io.loadmat(file_name=path, simplify_cells=True)['logger_aligned']}

loggerAligned_objs = {}
[loggerAligned_objs.update(load_loggerAligned(path)) for path in paths_local_loggerAligned];

['/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230112/logger_aligned.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230113/logger_aligned.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230114/logger_aligned.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230115/logger_aligned.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230116/logger_aligned.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230117/logger_aligned.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230118/logger_aligned.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230119/logger_aligned.mat']

In [22]:
paths_local_expParams = bnpm.path_helpers.find_paths(
    dir_outer=f'/media/rich/bigSSD/downloads_tmp/tmp_data/{mouse}/loggers/',
    reMatch='expParams.mat',
    depth=2
)
paths_local_expParams = [p for p in paths_local_expParams if Path(p).parent.name in dates_toUse]
display(paths_local_expParams)

load_expParams = lambda path: {Path(path).parent.name: scipy.io.loadmat(file_name=path, simplify_cells=True)['expParams']}

expParams_objs = {}
[expParams_objs.update(load_expParams(path)) for path in paths_local_expParams];

['/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230112/expParams.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230113/expParams.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230114/expParams.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230115/expParams.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230116/expParams.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230117/expParams.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230118/expParams.mat',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/loggers/20230119/expParams.mat']

In [23]:
loggerAligned_dfs = {
    date: {
        key_outer: pd.DataFrame(
            data=loggerAligned_objs[date][key_outer][:, :len(expParams_objs[date]['loggerNames'][key_outer])], 
            columns=expParams_objs[date]['loggerNames'][key_outer],
        ) for key_outer in expParams_objs[date]['loggerNames'].keys()
    } for date in expParams_objs.keys()
}

In [24]:
# n_sessions = len(loggerAligned_dfs)

# trials_duration_frame = []
# trials_duration_nowTime = []
# trials_success = []
# trials_type_111 = []

# for logger_df in loggerAligned_dfs.values():
#     ## pull out frames of start and end. -1 to account for matlab 1-indexing
#     frame_start__idx_logger = logger_df['trials']['counter_frameNum_trialStart'].to_numpy() - 1
#     frame_end__idx_logger = logger_df['trials']['counter_frameNum_trialEnd'].to_numpy() - 1   
    
#     ## pull out durations using time
#     time_start__idx_logger = logger_df['trials']['time_now_trialStart'].to_numpy()
#     time_end__idx_logger = logger_df['trials']['time_now_trialEnd'].to_numpy()
#     trials_duration_nowTime.append((time_end__idx_logger - time_start__idx_logger))

#     ## pull out whether the outcome was a success
#     trials_success.append(logger_df['trials']['success_outcome'] == 1)
    
#     ## pull out whether the trial type was a standard trial with feedback, reward, and cursor all one (1 1 1)
#     trials_type_111.append((logger_df['trials']['trialType_cursorOn'] == 1) * (logger_df['trials']['trialType_feedbackLinked'] == 1) * (logger_df['trials']['trialType_rewardOn'] == 1))
    

In [25]:
def get_frames_111_success(logger):
    frames_trialEnds   = logger['trials']['counter_frameNum_trialEnd'].to_numpy()
    frames_trialStarts = logger['trials']['counter_frameNum_trialStart'].to_numpy()

    type_111_success = (logger['trials']['trialType_cursorOn'] * logger['trials']['trialType_feedbackLinked'] * logger['trials']['trialType_rewardOn'] * logger['trials']['success_outcome']).to_numpy()
    framesEnd_111_success = frames_trialEnds[type_111_success[:len(frames_trialEnds)].astype(bool)]
    framesStart_111_success = frames_trialStarts[type_111_success[:len(frames_trialStarts)].astype(bool)]
    
    out = {
        'trialNum': logger['trials']['trialNum_trialStart'],
        'frames_start': framesStart_111_success,
        'frames_end': framesEnd_111_success,
        'duration': logger['trials']['counter_frameNum_trialEnd'] - logger['trials']['counter_frameNum_trialStart'],
    }
    
    return out

trials_111_success = {date: get_frames_111_success(logger) for date, logger in loggerAligned_dfs.items()}

In [26]:
import ScanImageTiffReader

In [27]:
paths_videos = bnpm.path_helpers.find_paths(str(Path('/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/') / date), reMatch='exp_', depth=2)[:]
# paths_videos = bnpm.path_helpers.find_paths('/media/rich/bigHDD/mouse_1511L/20230119/', reMatch='exp_', depth=2)[:]

display(paths_videos)

['/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/20230119/exp/exp_00001_00001.tif',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/20230119/exp/exp_00001_00002.tif',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/20230119/exp/exp_00001_00003.tif',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/20230119/exp/exp_00001_00004.tif',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/20230119/exp/exp_00001_00005.tif',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/20230119/exp/exp_00001_00006.tif',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/20230119/exp/exp_00001_00007.tif',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/20230119/exp/exp_00001_00008.tif',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L/tmp_scanimage/20230119/exp/exp_00001_00009.tif',
 '/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_1511L

In [28]:
triggers_start = trials_111_success[date]['frames_start']
triggers_end = trials_111_success[date]['frames_end']

In [29]:
frames_trs = np.cumsum([0] + [1000 for t in paths_videos])

In [30]:
xShifts = np.nan_to_num(loggerAligned_dfs[date]['motionCorrection']['xShift'].to_numpy()).astype(int)
yShifts = np.nan_to_num(loggerAligned_dfs[date]['motionCorrection']['yShift'].to_numpy()).astype(int)

xShifts_blocks = list(bnpm.indexing.make_batches(xShifts, batch_size=1000))
yShifts_blocks = list(bnpm.indexing.make_batches(yShifts, batch_size=1000))

In [31]:
def get_frames(path, xs=None, ys=None):
    tr = ScanImageTiffReader.ScanImageTiffReader(path)
#     if xs is not None:
#         f = bnpm.image_processing.apply_shifts_along_axis(tr.data(), xShifts=xs, yShifts=ys, workers=1)
#     else:
    f = tr.data()
    tr.close()
    del tr
    gc.collect()
    gc.collect()
#     return f
    return bnpm.image_processing.bin_array(f, [1,2,2], function=partial(np.mean, dtype=f.dtype))
frames = bnpm.parallel_helpers.map_parallel(get_frames, [paths_videos[:], xShifts_blocks[:], yShifts_blocks[:]], workers=-1, method='multithreading')
# frames = []
# for ii, (tr,x,y) in tqdm(enumerate(zip(paths_videos, xShifts_blocks, yShifts_blocks)), total=len(paths_videos)):
#     frames.append(get_frames(tr, x, y))
#     if ii >= 75:
#         break

100%|████████████████████████████████████████████████████████████████████████████| 108/108 [11:47<00:00,  6.55s/it]


In [32]:
frames_cat = torch.cat([torch.as_tensor(f, dtype=torch.int16) for f in frames[:]], dim=0)

In [ ]:
del frames
gc.collect()

In [34]:
FOV_baseline = bnpm.timeSeries.rolling_percentile_pd(frames_cat[np.linspace(0,108000,1000, endpoint=False, dtype=np.int64)].numpy().reshape(1000, frames_cat.shape[1]*frames_cat.shape[2]).T, ptile=30, window=100).T.reshape(1000, frames_cat.shape[1], frames_cat.shape[2])

In [35]:
FOV_decoder = expParams_objs[date]['baselineStuff']['ROIs']['cellWeightings']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
FOV_baseline_interp = torch.nn.functional.interpolate(torch.as_tensor(FOV_baseline, dtype=torch.float32).permute(1,2,0), scale_factor=108, mode='linear').permute(2,0,1).type(torch.int16).numpy()

In [ ]:
frames_df = (torch.as_tensor(frames_cat) - torch.as_tensor(FOV_baseline_interp)).numpy()

In [ ]:
# del FOV_baseline_interp
gc.collect()

In [ ]:
bnpm.plotting_helpers.display_toggle_image_stack(frames_df)

In [ ]:
et_traces_start = bnpm.timeSeries.event_triggered_traces(frames_df, triggers_start[:], win_bounds=[-10,0], dim=0)[0].mean(-1).astype(np.int16)
et_traces_end   = bnpm.timeSeries.event_triggered_traces(frames_df, triggers_end[:], win_bounds=[-10,0], dim=0)[0].mean(-1).astype(np.int16)

In [ ]:
import sklearn

lr = sklearn.linear_model.LogisticRegression(
    penalty='none',
    dual=False, 
    tol=0.0001, 
    C=0.1, 
    fit_intercept=True, 
    intercept_scaling=1, 
    class_weight=None, 
    random_state=None, 
    solver='lbfgs', 
    max_iter=100, 
    multi_class='auto', 
    verbose=0, 
    warm_start=False, 
    n_jobs=None, 
    l1_ratio=None
)


In [ ]:
import scipy.stats

In [ ]:
X = np.concatenate((et_traces_end.transpose(2,0,1).reshape(et_traces_end.shape[2], -1), et_traces_start.transpose(2,0,1).reshape(et_traces_start.shape[2], -1)), axis=0)
# X = scipy.stats.zscore(X, axis=0)
y = np.concatenate((np.ones(et_traces_end.shape[2]), np.zeros(et_traces_start.shape[2]))).astype(np.int64)
lr.fit(X, y)
lr.fit(X=scipy.stats.zscore(X, axis=0), y=y)

In [ ]:
import bnpm.similarity
sim_cov = bnpm.similarity.pairwise_similarity(v1=X, v2=y[:,None], method='cov')
sim_cc  = bnpm.similarity.pairwise_similarity(v1=X, v2=y[:,None], method='pearson')

In [ ]:
coef_cov = sim_cov.squeeze().reshape(256, 512)
coef_cc  =  sim_cc.squeeze().reshape(256, 512)

In [ ]:
coef_lr = lr.coef_.squeeze().reshape(256, 512)

In [ ]:
plt.figure()
plt.imshow(coef_cov, cmap='bwr')
plt.colorbar()

In [ ]:
plt.figure()
plt.imshow(coef_cc, cmap='bwr')
plt.colorbar()

In [ ]:
plt.figure()
plt.imshow(coef_lr, cmap='bwr')
plt.colorbar()

In [ ]:
bnpm.file_helpers.pickle_save(
    obj={
        'date': date,
        'mouse': mouse,
        'cage': cage,
        'FOV_decoder': FOV_decoder,
        'coef_cov': coef_cov,
        'coef_lr': coef_lr,
        'coef_cc': coef_cc,
        'et_traces_end': et_traces_end,
        'et_traces_start': et_traces_start,
        'X': X,
        'y': y,
    },
    path_save=f'/home/rich/Desktop/pixelwise/pix_{date}.pkl',
    mkdir=True,
)